In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
company_dataset = pd.read_csv("Data/CRSP_stocks_clean.csv")

/var/folders/9x/zgzyj9l153v5qjjz_hdvd31r0000gn/T/ipykernel_28555/877988600.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  company_dataset = pd.read_csv("Data/CRSP_stocks_clean.csv")


In [3]:
company_dataset[company_dataset["TICKER"]=="AA"]

,Unnamed: 0,date,TICKER,CUSIP,PRC,RET,VOL,BID,ASK,SHROUT
636697,887,2000-01-03,AA,44320110,80.9375,-0.024849,1551299.0,80.750,81.125,366407.0
636698,9421,2000-01-04,AA,44320110,81.3125,0.004633,2234799.0,81.125,81.500,366407.0
636699,17954,2000-01-05,AA,44320110,86.0000,0.057648,3121599.0,85.750,86.250,366407.0
636700,26490,2000-01-06,AA,44320110,84.8750,-0.013081,4494699.0,84.625,85.125,366407.0
636701,35023,2000-01-07,AA,44320110,84.6250,-0.002946,4534699.0,85.000,85.375,366407.0
...,...,...,...,...,...,...,...,...,...,...
640927,30094608,2016-10-25,AA,44320110,27.2900,0.016009,9682861.0,27.290,27.300,438460.0
640928,30101849,2016-10-26,AA,44320110,27.7900,0.018322,7794174.0,27.790,27.800,438460.0
640929,30109093,2016-10-27,AA,44320110,28.2200,0.015473,9271414.0,28.210,28.220,438460.0
640930,30116338,2016-10-28,AA,44320110,28.3700,0.005315,8937600.0,28.360,28.370,438460.0


In [4]:
#to change when we have more recent data
today = pd.to_datetime(date(2022, 12, 30))
#create historical mean starting date, should be about 3 years from current
historicalstart = pd.to_datetime(date(2019,12,30))

In [5]:
testdf = company_dataset
testdf = testdf[testdf['TICKER']=='AAPL']
print(testdf.dtypes)

Unnamed: 0      int64
date           object
TICKER         object
CUSIP          object
PRC           float64
RET           float64
VOL           float64
BID           float64
ASK           float64
SHROUT        float64
dtype: object


In [8]:
def ConditionalMean(returns,startdate):
    #filter rows where date is after startdate
    conditionalMeanDF = returns[returns['date'] > startdate]
    #take the mean return
    conditionalmean = conditionalMeanDF["RET"].mean()
    return conditionalmean

def VolatilityDF(returns):
   #Calculate standard deviation of returns
    volatilitydf = returns
    volatilitydf.set_index('date',inplace=True)
    volatilitydf["Std"] = volatilitydf.rolling(window=22).std()
    volatilitydf = volatilitydf.reset_index()
    return(volatilitydf)

In [9]:
# groupy by ticker
grouped_data = company_dataset.groupby('TICKER')
returns = pd.DataFrame({})
for ticker,data in grouped_data:
    #creating a returns dataframe for each ticker with return and date columns, then indexing by date
    print(ticker)
    returns = data[['RET','date']].dropna()
    returns['date'] = pd.to_datetime(returns['date'])
    returns.set_index('date',inplace=True)
    returns = returns.rolling(window=22).mean()
    returns = returns.reset_index()
    print(ConditionalMean(returns,historicalstart))
    print(VolatilityDF(returns))

A
0.00101995172330971
           date       RET       Std
0    2000-01-03       NaN       NaN
1    2000-01-04       NaN       NaN
2    2000-01-05       NaN       NaN
3    2000-01-06       NaN       NaN
4    2000-01-07       NaN       NaN
...         ...       ...       ...
5782 2022-12-23 -0.002142  0.002609
5783 2022-12-27 -0.001607  0.002719
5784 2022-12-28 -0.002522  0.002859
5785 2022-12-29 -0.000251  0.002908
5786 2022-12-30 -0.000211  0.002934

[5787 rows x 3 columns]
AA
nan
           date       RET       Std
0    2000-01-03       NaN       NaN
1    2000-01-04       NaN       NaN
2    2000-01-05       NaN       NaN
3    2000-01-06       NaN       NaN
4    2000-01-07       NaN       NaN
...         ...       ...       ...
4230 2016-10-25 -0.002786  0.001711
4231 2016-10-26 -0.001580  0.001711
4232 2016-10-27 -0.000736  0.001708
4233 2016-10-28 -0.001578  0.001701
4234 2016-10-31 -0.001385  0.001702

[4235 rows x 3 columns]
AAIC
0.0003236861144526549
          date       RET      